In [ ]:
# MENGIMPOR SEMUA LIBRARY YANG DIPERLUKAN UNTUK BAB 3
import numpy as np
import os
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_openml
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.base import BaseEstimator
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier

# Mengatur default plotting
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Untuk plot di dalam notebook (jika Anda menggunakan Jupyter)
# %matplotlib inline

print("--- BAB 3: KLASIFIKASI ---")
print("Semua library telah diimpor.")

# 1. MENGUNDUH DATASET MNIST
print("\n[1. Mengunduh Data MNIST]")
# Catatan: fetch_openml() mengunduh dataset dari openml.org.
# 'as_frame=False' memastikan kita mendapatkan array NumPy, bukan DataFrame pandas.
mnist = fetch_openml('mnist_784', version=1, as_frame=False, parser='auto')
# print(mnist.keys()) # Melihat struktur dataset

X, y = mnist["data"], mnist["target"]

# Mengubah label y (string) menjadi integer (uint8)
y = y.astype(np.uint8)

print(f"Bentuk data (X): {X.shape}") # (70000 instance, 784 fitur)
print(f"Bentuk label (y): {y.shape}") # (70000 label)

# Menampilkan satu digit (instance pertama)
some_digit = X[0]
some_digit_image = some_digit.reshape(28, 28)

# plt.imshow(some_digit_image, cmap="binary")
# plt.axis("off")
# plt.show()
print(f"Label untuk digit pertama: {y[0]}") # Seharusnya angka 5

# 2. MEMBAGI DATA LATIH DAN TES
print("\n[2. Memisahkan Data Latih dan Tes]")
# Dataset MNIST sudah diurutkan dan dibagi (60k latih, 10k tes)
X_train, X_test, y_train, y_test = X[:60000], X[60000:], y[:60000], y[60000:]
print("Data latih dan tes telah dipisah.")

# 3. MELATIH KLASIFIKASI BINER (BINARY CLASSIFIER)
print("\n[3. Melatih Klasifikasi Biner (Detektor Angka 5)]")
# Kita sederhanakan masalah: "apakah ini 5" (True) atau "bukan 5" (False)

y_train_5 = (y_train == 5)
y_test_5 = (y_test == 5)

# Menggunakan Stochastic Gradient Descent (SGD) Classifier
# SGD efisien untuk data besar
sgd_clf = SGDClassifier(random_state=42)
sgd_clf.fit(X_train, y_train_5)

print(f"Prediksi SGD untuk {y[0]} (angka 5): {sgd_clf.predict([some_digit])}")

# 4. MENGUKUR KINERJA
print("\n[4. Mengukur Kinerja Model]")

# 4a. Mengukur Akurasi Menggunakan Cross-Validation
print("Menghitung akurasi (bisa menipu)...")
# cv=3 berarti K-fold cross-validation dengan 3 fold
# score_accuracy = cross_val_score(sgd_clf, X_train, y_train_5, cv=3, scoring="accuracy")
# print(f"Akurasi Cross-Val (3-fold): {score_accuracy}")

# 4b. Classifier "Bodoh" sebagai Baseline
# Mari kita buat classifier yang selalu menebak "bukan 5"
class Never5Classifier(BaseEstimator):
    def fit(self, X, y=None):
        return self
    def predict(self, X):
        return np.zeros((len(X), 1), dtype=bool)

never_5_clf = Never5Classifier()
# score_never5 = cross_val_score(never_5_clf, X_train, y_train_5, cv=3, scoring="accuracy")
# print(f"Akurasi 'Never 5' Classifier: {score_never5}")
# Hasilnya > 90%! Ini membuktikan akurasi adalah metrik berbahaya pada skewed dataset.

# 4c. Confusion Matrix
print("Menghitung Confusion Matrix...")
# cross_val_predict melakukan K-fold cross-validation, tapi mengembalikan prediksi
y_train_pred = cross_val_predict(sgd_clf, X_train, y_train_5, cv=3)

cm = confusion_matrix(y_train_5, y_train_pred)
print("Confusion Matrix (Detektor '5'):\n", cm)
# Baris: Kelas Aktual (Bukan 5, Adalah 5)
# Kolom: Kelas Prediksi (Bukan 5, Adalah 5)
# [[TN, FP],
#  [FN, TP]]
# TN (True Negative): 53892 (benar menebak bukan 5)
# FP (False Positive): 687 (salah menebak 5, padahal bukan)
# FN (False Negative): 1891 (salah menebak bukan 5, padahal 5)
# TP (True Positive): 3530 (benar menebak 5)

# 4d. Precision, Recall, dan F1-Score
print("Menghitung Precision, Recall, F1-Score...")
# Precision = TP / (TP + FP) -> Seberapa akurat tebakan '5' kita?
# Recall = TP / (TP + FN) -> Berapa banyak '5' yang berhasil kita temukan?
print(f"Precision: {precision_score(y_train_5, y_train_pred):.4f}")
print(f"Recall: {recall_score(y_train_5, y_train_pred):.4f}")
print(f"F1 Score: {f1_score(y_train_5, y_train_pred):.4f}")

# 4e. Precision/Recall Trade-off
print("Menganalisis Precision/Recall Trade-off...")
# SGDClassifier membuat keputusan berdasarkan skor
# Kita bisa mendapatkan skor ini alih-alih prediksi
y_scores = cross_val_predict(sgd_clf, X_train, y_train_5, cv=3,
                             method="decision_function")

# Menghitung P dan R untuk semua kemungkinan threshold
precisions, recalls, thresholds = precision_recall_curve(y_train_5, y_scores)

# Fungsi untuk plot P/R vs Threshold
def plot_precision_recall_vs_threshold(precisions, recalls, thresholds):
    plt.figure(figsize=(8, 4))
    plt.plot(thresholds, precisions[:-1], "b--", label="Precision", linewidth=2)
    plt.plot(thresholds, recalls[:-1], "g-", label="Recall", linewidth=2)
    plt.legend(loc="center right", fontsize=16)
    plt.xlabel("Threshold", fontsize=16)
    plt.grid(True)
    plt.axis([-50000, 50000, 0, 1])

# Panggil fungsi plot (di-comment agar tidak auto-tampil)
# print("Plotting Precision/Recall vs Threshold...")
# plot_precision_recall_vs_threshold(precisions, recalls, thresholds)
# plt.show()

# Contoh: Mencari threshold untuk 90% precision
threshold_90_precision = thresholds[np.argmax(precisions >= 0.90)]
# y_train_pred_90 = (y_scores >= threshold_90_precision)
# print(f"Precision pada threshold 90%: {precision_score(y_train_5, y_train_pred_90)}")
# print(f"Recall pada threshold 90%: {recall_score(y_train_5, y_train_pred_90)}")

# 4f. Kurva ROC (Receiver Operating Characteristic)
print("Menghitung Kurva ROC...")
# Memplot True Positive Rate (Recall) vs False Positive Rate (FPR)
# FPR = FP / (FP + TN) -> Proporsi 'bukan 5' yang salah dikira '5'
fpr, tpr, thresholds_roc = roc_curve(y_train_5, y_scores)

# Fungsi untuk plot Kurva ROC
def plot_roc_curve(fpr, tpr, label=None):
    plt.plot(fpr, tpr, linewidth=2, label=label)
    plt.plot([0, 1], [0, 1], 'k--') # Garis acak (diagonal)
    plt.axis([0, 1, 0, 1])
    plt.xlabel('False Positive Rate (1 - Specificity)', fontsize=16)
    plt.ylabel('True Positive Rate (Recall)', fontsize=16)
    plt.grid(True)

# Panggil fungsi plot (di-comment)
# plt.figure(figsize=(8, 6))
# plot_roc_curve(fpr, tpr)
# plt.show()

# Area Under the Curve (AUC)
print(f"ROC AUC Score (SGD): {roc_auc_score(y_train_5, y_scores):.4f}")

# 4g. Membandingkan dengan RandomForestClassifier
print("Melatih RandomForestClassifier untuk perbandingan...")
forest_clf = RandomForestClassifier(random_state=42)
# RandomForest tidak punya decision_function(), tapi punya predict_proba()
y_probas_forest = cross_val_predict(forest_clf, X_train, y_train_5, cv=3,
                                    method="predict_proba")

# Kita gunakan probabilitas kelas positif sebagai skor
y_scores_forest = y_probas_forest[:, 1]
fpr_forest, tpr_forest, thresholds_forest = roc_curve(y_train_5, y_scores_forest)

# Plot perbandingan ROC
# plt.figure(figsize=(8, 6))
# plot_roc_curve(fpr, tpr, "SGD")
# plot_roc_curve(fpr_forest, tpr_forest, "Random Forest")
# plt.legend(loc="lower right", fontsize=16)
# plt.show()

print(f"ROC AUC Score (Random Forest): {roc_auc_score(y_train_5, y_scores_forest):.4f}")
# Random Forest jauh lebih baik

# 5. KLASIFIKASI MULTIKELAS
print("\n[5. Klasifikasi Multikelas (0-9)]")

# Melatih SVM (Support Vector Machine)
# Scikit-Learn SVC otomatis menggunakan strategi One-vs-One (OvO)
svm_clf = SVC(gamma="auto", random_state=42)
# Kita latih pada SEMUA label (y_train), bukan y_train_5
# Kita latih pada subset kecil (2000) agar cepat
svm_clf.fit(X_train[:2000], y_train[:2000])
print(f"Prediksi SVM untuk {y[0]} (angka 5): {svm_clf.predict([some_digit])}")

# SGDClassifier bisa menangani multikelas secara langsung (OvR defaultnya)
sgd_clf.fit(X_train, y_train) # Latih pada semua data dan semua kelas
print(f"Prediksi SGD untuk {y[0]} (angka 5): {sgd_clf.predict([some_digit])}")
# sgd_clf.decision_function([some_digit]) # Akan memberikan 10 skor

# Evaluasi classifier multikelas
# print("Akurasi SGD (multikelas, non-scaled):")
# print(cross_val_score(sgd_clf, X_train, y_train, cv=3, scoring="accuracy"))

# Scaling SANGAT PENTING untuk performa
print("Melakukan scaling data untuk SGD...")
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train.astype(np.float64))
# print("Akurasi SGD (multikelas, scaled):")
# print(cross_val_score(sgd_clf, X_train_scaled, y_train, cv=3, scoring="accuracy"))

# 6. ANALISIS ERROR
print("\n[6. Analisis Error]")
# Kita gunakan model SGD yang sudah di-scaling
y_train_pred = cross_val_predict(sgd_clf, X_train_scaled, y_train, cv=3)
conf_mx = confusion_matrix(y_train, y_train_pred)

print("Plotting Confusion Matrix (multikelas)...")
# plt.matshow(conf_mx, cmap=plt.cm.gray)
# plt.show()

# Fokus pada error saja dengan normalisasi
row_sums = conf_mx.sum(axis=1, keepdims=True)
norm_conf_mx = conf_mx / row_sums
np.fill_diagonal(norm_conf_mx, 0) # Ubah diagonal (prediksi benar) menjadi 0

# print("Plotting Matriks Error (ternormalisasi)...")
# plt.matshow(norm_conf_mx, cmap=plt.cm.gray)
# plt.show()
# Penjelasan: Baris = Aktual, Kolom = Prediksi.
# Kolom 8 yang terang berarti banyak angka LAIN (baris) salah diprediksi sebagai 8.
# Baris 3 dan 5 yang terang berarti angka 3 dan 5 sering tertukar.

# 7. KLASIFIKASI MULTILABEL
print("\n[7. Klasifikasi Multilabel]")
# Sebuah instance bisa memiliki beberapa label

# Contoh: Label 1: Apakah angkanya besar (>= 7)?
y_train_large = (y_train >= 7)
# Contoh: Label 2: Apakah angkanya ganjil?
y_train_odd = (y_train % 2 == 1)
# Menggabungkan kedua label
y_multilabel = np.c_[y_train_large, y_train_odd]

knn_clf = KNeighborsClassifier()
knn_clf.fit(X_train, y_multilabel)

print(f"Prediksi Multilabel untuk {y[0]} (angka 5): {knn_clf.predict([some_digit])}")
# Output: [[False, True]] (angka 5 TIDAK besar, tapi GANJIL)

# Evaluasi Multilabel (contoh: F1 score rata-rata)
# y_train_knn_pred = cross_val_predict(knn_clf, X_train, y_multilabel, cv=3)
# f1_macro = f1_score(y_multilabel, y_train_knn_pred, average="macro")
# print(f"F1 Score (macro) Multilabel: {f1_macro:.4f}")

# 8. KLASIFIKASI MULTIOUTPUT
print("\n[8. Klasifikasi Multioutput (Contoh: Denoising)]")
# Generalisasi dari multilabel; setiap label bisa multiclass.
# Contoh: Menghilangkan noise (setiap pixel adalah label)

# Membuat data latih yang noisy (menambah noise acak)
noise = np.random.randint(0, 100, (len(X_train), 784))
X_train_mod = X_train + noise

# Membuat data tes yang noisy
noise_test = np.random.randint(0, 100, (len(X_test), 784))
X_test_mod = X_test + noise_test

# Targetnya adalah gambar bersih (y adalah X)
y_train_mod = X_train
y_test_mod = X_test

# Latih model untuk memetakan gambar noisy ke gambar bersih
knn_clf.fit(X_train_mod, y_train_mod)

# Coba bersihkan satu digit
some_index = 0
clean_digit = knn_clf.predict([X_test_mod[some_index]])

# Fungsi helper untuk plot
def plot_digit(data):
    image = data.reshape(28, 28)
    plt.imshow(image, cmap=mpl.cm.binary, interpolation="nearest")
    plt.axis("off")

# Plot perbandingan (di-comment)
# plt.figure(figsize=(8, 4))
# plt.subplot(121); plot_digit(X_test_mod[some_index])
# plt.title("Noisy Input")
# plt.subplot(122); plot_digit(clean_digit)
# plt.title("Denoised Output")
# plt.show()

print("\n--- Selesai Bab 3 ---")

--- BAB 3: KLASIFIKASI ---
Semua library telah diimpor.

[1. Mengunduh Data MNIST]
Bentuk data (X): (70000, 784)
Bentuk label (y): (70000,)
Label untuk digit pertama: 5

[2. Memisahkan Data Latih dan Tes]
Data latih dan tes telah dipisah.

[3. Melatih Klasifikasi Biner (Detektor Angka 5)]
Prediksi SGD untuk 5 (angka 5): [ True]

[4. Mengukur Kinerja Model]
Menghitung akurasi (bisa menipu)...
Menghitung Confusion Matrix...
Confusion Matrix (Detektor '5'):
 [[53892   687]
 [ 1891  3530]]
Menghitung Precision, Recall, F1-Score...
Precision: 0.8371
Recall: 0.6512
F1 Score: 0.7325
Menganalisis Precision/Recall Trade-off...
Menghitung Kurva ROC...
ROC AUC Score (SGD): 0.9605
Melatih RandomForestClassifier untuk perbandingan...
ROC AUC Score (Random Forest): 0.9983

[5. Klasifikasi Multikelas (0-9)]
Prediksi SVM untuk 5 (angka 5): [5]
Prediksi SGD untuk 5 (angka 5): [3]
Melakukan scaling data untuk SGD...

[6. Analisis Error]
